In [1]:
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
import torch.utils.data as data

In [19]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import io
import os
import sys
import json
import urllib3
import multiprocessing

from PIL import Image
from tqdm import tqdm
from urllib3.util import Retry

In [56]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 3.2MB/s ta 0:00:011
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Download data

In [63]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=90)


def parse_dataset(_dataset, _outdir, _max=10000):
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
    return _fnames_urls[:_max]

In [ ]:
fnames_and_urls = []
url = image["url"]
fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
_fnames_urls.append((fname, url))

## Load data

In [78]:
with open("/Users/jiayi/1008 deep learning/data/train.json") as json_data:
    train = json.load(json_data)
with open('/Users/jiayi/1008 deep learning/data/test.json') as json_data:
    test= json.load(json_data)
with open('/Users/jiayi/1008 deep learning/data/validation.json') as json_data:
    validation = json.load(json_data)

In [79]:
# JSON TO PANDAS DATAFRAME
# train data
train_img_url=train['images']
train_img_url=pd.DataFrame(train_img_url)
train_ann=train['annotations']
train_ann=pd.DataFrame(train_ann)
train=pd.merge(train_img_url, train_ann, on='imageId', how='inner')

In [80]:
# Validation Data (total 9897)
val_img_url=validation['images']
val_img_url=pd.DataFrame(val_img_url)
val_ann=validation['annotations']
val_ann=pd.DataFrame(val_ann)
validation=pd.merge(val_img_url, val_ann, on='imageId', how='inner')

In [81]:
# test data
test=pd.DataFrame(test['images'])

In [75]:
test.head()

,imageId,url
0,1,https://contestimg.wish.com/api/webimage/568e1...
1,2,https://contestimg.wish.com/api/webimage/5452f...
2,3,https://contestimg.wish.com/api/webimage/54058...
3,4,https://contestimg.wish.com/api/webimage/540c6...
4,5,https://contestimg.wish.com/api/webimage/54477...


### Multilabel Binarizer

In [82]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_label = mlb.fit_transform(train['labelId'])
validation_label = mlb.transform(validation['labelId'])
dummy_label_col = list(mlb.classes_)

In [97]:
# convert train_label to dataframe
train_label = pd.DataFrame(data = train_label, columns = list(mlb.classes_))
# train_label.head()
validation_label = pd.DataFrame(data = validation_label, columns = list(mlb.classes_))
# validation_label.head()

In [95]:
# take first 10000 images as smaller training data and first 200 as val data
train_sm = train[:10000]
train_sm_label = train_label[:10000]
val_sm = validation[:200]
val_sm_label = validation_label[:200]

In [106]:
train_sm_label.iloc[0,:].as_matrix().reshape(-1).shape

(228,)

In [107]:
test_label = pd.read_csv('./data/train_label_col.csv')

In [109]:
test_label.shape

(10000, 228)

In [98]:
# Save as numpy
# dummy_label_col = pd.DataFrame(columns = dummy_label_col)
# dummy_label_col.to_csv('/Users/jiayi/1008 deep learning/data/dummy_label_col.csv', index = False)
# train_sm_label.to_csv('/Users/jiayi/1008 deep learning/data/train_label_col.csv', index = False)
# val_sm_label.to_csv('/Users/jiayi/1008 deep learning/data/val_label_col.csv', index = False)

# np.save('%s/dummy_label_train.npy' % '', train_label)
# np.save('%s/dummy_label_val.npy' % '', validation_label)
# dummy_label_col.head()

### Convert data to Dataloader 

In [5]:
class FashionDataset(data.Dataset):

    def __init__(self, root, csv_file, transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.label_file = pd.read_csv(csv_file)

    def __getitem__(self, index):
        img_path = os.path.join(self.root,"{}.jpg".format(index))
        image = Image.open(img_path)
        label = self.label_file.iloc[index,:].as_matrix()

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.label_file)

In [29]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [30]:
train_dataset=FashionDataset(root='/Users/jiayi/1008 deep learning/Image/train_resize',csv_file='/Users/jiayi/1008 deep learning/data/train_label_col.csv',transform=data_transform)
val_dataset=FashionDataset(root='/Users/jiayi/1008 deep learning/Image/val_resize',csv_file='/Users/jiayi/1008 deep learning/data/val_label_col.csv',transform=data_transform)

In [125]:
for i in range(10):
    img,label = train_dataset.__getitem__(i)
    print(img,label)


( 0 ,.,.) = 
  0.5412  0.5686  0.5373  ...   0.5059  0.5451  0.5647
  0.3098  0.3176  0.2431  ...   0.5020  0.5412  0.5608
  0.1725  0.1647  0.0863  ...   0.4902  0.5255  0.5451
           ...             ⋱             ...          
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745

( 1 ,.,.) = 
  0.5804  0.6078  0.5647  ...   0.5333  0.5725  0.5922
  0.3490  0.3451  0.2706  ...   0.5333  0.5725  0.5922
  0.2000  0.1922  0.1137  ...   0.5451  0.5804  0.6000
           ...             ⋱             ...          
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784

( 2 ,.,.) = 
  0.5490  0.5765  0.5373  ...   0.4706  0.5098  0.5294
  0.3176  0.3176  0.2431  ...   0.4745  0.5137  0.5333
  0.1725  0.1647  0.0863  ...   0.4863  0.5216  0.5412
           ...         


( 0 ,.,.) = 
  0.5412  0.5686  0.5373  ...   0.5059  0.5451  0.5647
  0.3098  0.3176  0.2431  ...   0.5020  0.5412  0.5608
  0.1725  0.1647  0.0863  ...   0.4902  0.5255  0.5451
           ...             ⋱             ...          
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745

( 1 ,.,.) = 
  0.5804  0.6078  0.5647  ...   0.5333  0.5725  0.5922
  0.3490  0.3451  0.2706  ...   0.5333  0.5725  0.5922
  0.2000  0.1922  0.1137  ...   0.5451  0.5804  0.6000
           ...             ⋱             ...          
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784

( 2 ,.,.) = 
  0.5490  0.5765  0.5373  ...   0.4706  0.5098  0.5294
  0.3176  0.3176  0.2431  ...   0.4745  0.5137  0.5333
  0.1725  0.1647  0.0863  ...   0.4863  0.5216  0.5412
           ...         

## CNN Model

In [31]:
batch_size=2
n_iters=5000
num_epochs=n_iters/(len(train_dataset)/batch_size)
num_epochs=int(num_epochs)
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
val_loader=torch.utils.data.DataLoader(dataset=val_dataset,batch_size=batch_size,shuffle=True)

In [32]:
class CNNModule(nn.Module):
    def __init__(self):
        super (CNNModule,self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=3,out_channels=30,kernel_size=9,stride=1)
        self.relu1=nn.ReLU()
        nn.init.xavier_uniform(self.cnn1.weight)


        self.maxpool1=nn.MaxPool2d(kernel_size=2)

        self.cnn2=nn.Conv2d(in_channels=30,out_channels=90,kernel_size=9,stride=1)
        self.relu2=nn.ReLU()
        nn.init.xavier_uniform(self.cnn2.weight)
        

        self.maxpool2=nn.MaxPool2d(kernel_size=2)

        self.fcl=nn.Linear(90*119*119,228)
        
        self.activation = nn.Sigmoid()
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
#         print ("CNN1")
#         print (out.size())
        
        out=self.maxpool1(out)
#         print ("Maxpool1")
#         print (out.size())
        
        out=self.cnn2(out)
        out=self.relu2(out)
#         print ("CNN2")
#         print (out.size())
        out=self.maxpool2(out)
#         print ("Maxpool2")
#         print (out.size(0))
        
        out=out.view(out.size(0),-1)

        out=self.fcl(out)
#         print(out.size())
        out = self.activation(out)
#         out[out>0.6] = 1
#         out[out<=0.6] = 0

        return out

In [34]:
model=CNNModule()
criterion=nn.CrossEntropyLoss()
learning_rate=0.015
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
iter=0
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate (train_loader):
        images=Variable(images)
        labels=Variable(labels)

        optimizer.zero_grad()
        outputs=model(images)
        print(outputs)
        print(labels)
#         out[out>0.6] = 1
#         out[out<=0.6] = 0
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        print(i,loss)
        break
        
#         iter+=1
#         if iter%500==0:
#             correct=0
#             total=0
#             for images,labels in val_loader:
#                 images=Variable(images)

#                 outputs=model(images)
                
#                 _,predicted=torch.max(outputs.data,1)
#                 total+=labels.size(0)
#                 correct+=(predicted==labels).sum()
#             accuracy= (100.0* correct)/(total)
#             print("Iteration:"+str(iter)+"  Loss:"+str(loss)+"  Accuracy:"+str(accuracy))


Variable containing:

Columns 0 to 9 
 0.5579  0.4462  0.4549  0.4113  0.5261  0.4440  0.5881  0.4423  0.5405  0.5925
 0.5350  0.4732  0.4688  0.4438  0.5067  0.4816  0.5824  0.4434  0.5667  0.5837

Columns 10 to 19 
 0.5631  0.5086  0.4681  0.5141  0.5182  0.4898  0.4918  0.4685  0.3709  0.4137
 0.5356  0.4929  0.4538  0.5181  0.5145  0.5205  0.5091  0.4816  0.3876  0.4217

Columns 20 to 29 
 0.5461  0.4971  0.3152  0.4731  0.4866  0.5262  0.4585  0.4952  0.5964  0.5383
 0.5398  0.5043  0.3652  0.4548  0.4805  0.5328  0.4602  0.5247  0.5713  0.5879

Columns 30 to 39 
 0.4855  0.5150  0.5299  0.4418  0.4616  0.3553  0.4832  0.3870  0.5558  0.4796
 0.4968  0.5270  0.5306  0.4335  0.4747  0.3903  0.5101  0.3891  0.5808  0.4903

Columns 40 to 49 
 0.4675  0.5042  0.4400  0.5612  0.5428  0.5180  0.4329  0.5522  0.5106  0.5378
 0.4537  0.5040  0.4498  0.5445  0.5599  0.4659  0.4581  0.5766  0.5101  0.5306

Columns 50 to 59 
 0.5412  0.6195  0.5711  0.4897  0.4632  0.3601  0.4917  0.4160  0.

RuntimeError: multi-target not supported at /Users/soumith/minicondabuild3/conda-bld/pytorch_1512381214802/work/torch/lib/THNN/generic/ClassNLLCriterion.c:22

In [17]:
outputs

Variable containing:

Columns 0 to 9 
 0.4930  0.4570  0.4742  0.5534  0.4965  0.5229  0.5253  0.5077  0.4365  0.4960

Columns 10 to 19 
 0.4361  0.4968  0.4941  0.5096  0.4907  0.4886  0.4777  0.4790  0.4819  0.5150

Columns 20 to 29 
 0.4811  0.5103  0.5058  0.5333  0.5107  0.4584  0.5181  0.5043  0.4742  0.4620

Columns 30 to 39 
 0.5015  0.5026  0.5687  0.4895  0.4902  0.4851  0.5093  0.5360  0.4789  0.4998

Columns 40 to 49 
 0.4968  0.5026  0.5016  0.5007  0.5031  0.5240  0.5218  0.5433  0.5129  0.5086

Columns 50 to 59 
 0.5191  0.5084  0.5095  0.5114  0.5032  0.4995  0.4581  0.4919  0.5283  0.4705

Columns 60 to 69 
 0.5269  0.5132  0.4981  0.4836  0.4992  0.4960  0.5201  0.4748  0.5170  0.4616

Columns 70 to 79 
 0.4890  0.4984  0.5084  0.4983  0.5033  0.4797  0.4524  0.5174  0.4932  0.5410

Columns 80 to 89 
 0.5115  0.5359  0.4390  0.5393  0.4860  0.4889  0.5116  0.5192  0.4977  0.5037

Columns 90 to 99 
 0.5287  0.5064  0.4465  0.4965  0.5105  0.5298  0.4682  0.5065  0.5301

## Resize image

In [127]:
for i in range(1,201):
    img_path="/Users/jiayi/1008 deep learning/Image/validation/{}.jpg".format(i)
    image = Image.open(img_path)
    tran_img = image.resize((500,500))
    tran_img.save("/Users/jiayi/1008 deep learning/Image/val_resize/{}.jpg".format(i), tran_img.format)
#     tensor_img = ToTensor()(tran_img).unsqueeze(0)
#     tensor_img = Variable(tensor_img)
# image = ToTensor()(image).unsqueeze(0) # unsqueeze to add artificial first dimension
# image = Variable(image)

In [37]:
img_path="/Users/jiayi/1008 deep learning/Image/train/1r.jpg"
image = Image.open(img_path)

In [40]:
tensor_image = ToTensor()(image).unsqueeze(0)

In [120]:
img_path="/Users/jiayi/1008 deep learning/Image/train_resize/{}.jpg".format(10)
image = Image.open(img_path)
img = ToTensor()(image)

In [121]:
img


( 0 ,.,.) = 
  0.5412  0.5686  0.5373  ...   0.5059  0.5451  0.5647
  0.3098  0.3176  0.2431  ...   0.5020  0.5412  0.5608
  0.1725  0.1647  0.0863  ...   0.4902  0.5255  0.5451
           ...             ⋱             ...          
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745
  0.9294  0.9412  0.9529  ...   0.8745  0.8745  0.8745

( 1 ,.,.) = 
  0.5804  0.6078  0.5647  ...   0.5333  0.5725  0.5922
  0.3490  0.3451  0.2706  ...   0.5333  0.5725  0.5922
  0.2000  0.1922  0.1137  ...   0.5451  0.5804  0.6000
           ...             ⋱             ...          
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784
  0.9059  0.9176  0.9294  ...   0.8784  0.8784  0.8784

( 2 ,.,.) = 
  0.5490  0.5765  0.5373  ...   0.4706  0.5098  0.5294
  0.3176  0.3176  0.2431  ...   0.4745  0.5137  0.5333
  0.1725  0.1647  0.0863  ...   0.4863  0.5216  0.5412
           ...         

In [31]:
a = np.asarray([2,1,12,3,1,4,2])
b = torch.from_numpy(a)
b.size()
b[b>3]=0
b


 2
 1
 0
 3
 1
 0
 2
[torch.LongTensor of size 7]

In [32]:
a

array([2, 1, 0, 3, 1, 0, 2])

In [33]:
c

3